# I. Tweet Data Cleaning



In [1]:
## Tweet Table

In [2]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json
import time
pd.set_option('display.max_rows', 5)

In [3]:
#function to cheack whether 
def check_var(data, x):
    try:
        data[x]
        return True
    except:
        return False

In [8]:
%%time

#making mongoDB database and collection
client = MongoClient('localhost', 27017)
db = client['mydatabase']

want_drop = input()
if(want_drop == 'yes'): ## if type yes, then it will drop tweet table from mongodb to prepare new importing.
    db.tweet.drop()
    
tweet = db['tweet']

no
CPU times: user 38.5 ms, sys: 17.7 ms, total: 56.2 ms
Wall time: 2min 14s


In [16]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [17]:
#yelin directory '/Users/yelin/Downloads/corona-out-3/corona-out-3'
#Mo 'C:/Users/maxmo/OneDrive/Desktop/Rutgers/Spring2020/Data Management/corona-out-2'
# Hui Wang  file = '/Users/huiwang/Downloads/corona-out-3'
file = '/Users/huiwang/Downloads/corona-out-3'

In [9]:
%%time

#id, user_id, created_at, text, retweet_count, favorite_count, reply_count, hashtags
import json
with open(file, 'r', encoding = 'utf8') as f1 :
    for line in f1 :
        try :
            data = json.loads(line)
        except :
            continue
        
        hashtag = [] #to collect list of hashtags
        
        if (data['lang'] == 'en'):
            
            #case 1: check whether it is a retweet data or not
            if (check_var(data, 'retweeted_status')) :

                Created_at = pd.to_datetime(data['retweeted_status']['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['retweeted_status']['user']['id']
                ID = data['retweeted_status']['id']
                retweet_count = data['retweeted_status']['retweet_count']
                favorite_count = data['retweeted_status']['favorite_count']
                reply_count = data['retweeted_status']['reply_count']

                #check whether there is extension version of that tweet or not        
                if (check_var(data['retweeted_status'], 'extended_tweet')) :
                    Text = data['retweeted_status']['extended_tweet']['full_text']
                    for t in data['retweeted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else :
                    Text = data['retweeted_status']['text']
                    for t in data['retweeted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

                #check whether it is retweet of quoted tweet or not
                if (check_var(data['retweeted_status'], 'quoted_status')) :

                    #check whether there is extension version of the original tweet or not 
                    if (check_var(data['retweeted_status']['quoted_status'], 'extended_tweet')) :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['extended_tweet']['full_text']
                        for t in data['retweeted_status']['quoted_status']['extended_tweet']['entities']['hashtags'] :
                            hashtag.append(t['text'])                
                    else :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['text']
                        for t in data['retweeted_status']['quoted_status']['entities']['hashtags'] :
                            hashtag.append(t['text'])               

            #case 2: check whether it is a quote tweet data or not
            elif (check_var(data, 'quoted_status')) :
                Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                #check whether there is extension version of that quote tweet or not 
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text'] + ' || ' 
                else:
                    Text = data['text'] + ' || '

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

                #check whether there is extension version of the originl tweet or not 
                if (check_var(data['quoted_status'], 'extended_tweet')) :
                    Text = Text + data['quoted_status']['extended_tweet']['full_text']
                    for t in data['quoted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else:
                    Text = Text + data['quoted_status']['text']
                    for t in data['quoted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

            #if it does not belong to case 1 and case 2, it is an ordinary tweet data
            else :
                Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                #check whether there is extension version of that tweet or not  
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text']
                else :
                    Text = data['text']

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

            tw = {'id' : ID,
                  'user_id' : UserID,
                  'created_at' : Created_at,
                  'text' : Text,
                  'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count,
                  'reply_count' : reply_count,
                  'hashtags': hashtag
                 }

            #if the data is already in our database, we update it 
            if (tweet.count_documents({'id' : tw['id']}) > 0) :
                db.tweet.update_one({'id' : tw['id']},
                                    {'$set' : {'retweet_count' : tw['retweet_count'],
                                               'favorite_count' : tw['favorite_count'],
                                               'reply_count' : tw['reply_count']}})
            #if it is not, we insert it    
            else :
                tweet.insert_one(tw)
            

tweet.create_index([ ('id', 1) ])

CPU times: user 1min 37s, sys: 5.64 s, total: 1min 43s
Wall time: 18min 50s


'id_1'

In [18]:
tweet_table = pd.DataFrame(tweet.find())
tweet_table.shape

(19076, 9)

In [19]:
tweet_table.head(5)

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea7785aa8a68922687753f6,1253949413191344128,207809313,"April 25, 2020, 07:30:12 AM",India’s war with Corona is ongoing.\n\nPlay yo...,398,2144,108,[FeedTheNeedy]
1,5ea7785aa8a68922687753f7,1253992905703862272,858859031464751104,"April 25, 2020, 10:23:01 AM",VIDEO | 25.04.20\n\n&gt; Monsta X appears on Y...,59,144,1,"[몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]"
2,5ea7785aa8a68922687753f8,1253697983343816706,2719753171,"April 24, 2020, 02:51:06 PM","Sri ByReddy Rajasekhar Reddy garu,a leader,who...",2238,7243,47,[]
3,5ea7785aa8a68922687753f9,1254022773598572544,1091660129894838272,"April 25, 2020, 12:21:42 PM",@VinceMcMahon @TripleH We hereby honor to anno...,0,0,0,[]
4,5ea7785aa8a68922687753fa,1254022776094105602,375777294,"April 25, 2020, 12:21:43 PM",im making 17 a hr doing nothing 😭😭😭😭 i love th...,0,0,0,[]






# II. User Data Cleaning





In [20]:
## User Table

In [21]:
import pandas as pd
import psycopg2
import json

In [27]:
##Creating the database
conn = psycopg2.connect("dbname=postgres port=5432 user=postgres password=password")
conn.set_session(autocommit = True)
cur = conn.cursor()

cur.execute("CREATE DATABASE twitter_users" )
conn.commit()
cur.close()
conn.close()

DuplicateDatabase: database "twitter_users" already exists


In [28]:
#Creating the table
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()
#cur.execute("drop table users_data")
cur.execute("CREATE TABLE IF NOT EXISTS users_data( user_id bigint PRIMARY KEY, name varchar(255), user_name varchar(255), verified_status boolean,followers_count integer,friends_count integer,statuses_count integer,user_location varchar(500),favourites_count integer);" )
conn.commit()
cur.close()
conn.close()

In [29]:
def check_var(data,x):
    try:
        data[x]
        return True
    except:
        return False

In [30]:
%%time

#This function takes the the variable inputs and stores it into a PostgreSQL database
def store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count):
    conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
    cur = conn.cursor()
    insert_query = "INSERT INTO users_data (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET verified_status=EXCLUDED.verified_status, followers_count=EXCLUDED.followers_count, friends_count=EXCLUDED.friends_count, statuses_count=EXCLUDED.statuses_count, user_location=EXCLUDED.user_location, favourites_count=EXCLUDED.favourites_count"
    cur.execute(insert_query, (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location,favourites_count))
    conn.commit()
    cur.close()
    conn.close()
    
    
# Decoding the JSON file
with open(file, 'r', encoding = 'utf8') as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
        except:
            continue
        
        if (data['lang'] == 'en'): # only 
            
            if (check_var(data, 'retweeted_status')):

                user_id = data['retweeted_status']['user']['id']
                name = data['retweeted_status']['user']['name']
                user_name = data['retweeted_status']['user']['screen_name']
                verified_status = data['retweeted_status']['user']['verified']
                followers_count = data['retweeted_status']['user']['followers_count']
                friends_count = data['retweeted_status']['user']['friends_count']
                statuses_count = data['retweeted_status']['user']['statuses_count']
                user_location = data['retweeted_status']['user']['location']
                favourites_count= data['retweeted_status']['user']['favourites_count']


            else :
                user_id = data['user']['id']
                name = data['user']['name']
                user_name = data['user']['screen_name']
                verified_status = data['user']['verified']
                followers_count = data['user']['followers_count']
                friends_count = data['user']['friends_count']
                statuses_count = data['user']['statuses_count']
                user_location = data['user']['location']
                favourites_count= data['user']['favourites_count']


            #insert the data into the PostgreSQL database
            store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count)

CPU times: user 40.5 s, sys: 9.62 s, total: 50.1 s
Wall time: 6min 31s


In [31]:
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()


# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [32]:
user_table = create_pandas_table("SELECT * FROM users_data")
tweet_table.shape

(19076, 9)

In [33]:
user_table.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,375777294,TeéLaneeë🌺,TWD40_,False,637,408,18976,None,1325
1,1132273796138905600,Terri Kamp,RampTheresa,False,1522,256,20861,None,29166
2,301470336,BABA ALI,_FreshAA,False,9406,1870,264287,"Chicago, IL",857
3,923169415054680064,Quantum,QuantumAspect,False,3196,1001,15219,The Netherlands,119803
4,95871559,Kapil,kapsology,True,152174,1282,230783,Delhi NCR,17302






# III. Searching Application





In [ ]:
##text search
user_keyword = input()
regx = re.compile(user_keyword, re.IGNORECASE)

search_tweet = pd.DataFrame(
    tweet.find({'text': {"$regex": regx}}).sort([("retweet_count", -1),("reply_count", -1),("favorite_count", -1)])
)

if(len(search_tweet)==0):
    print("Error: No result for %s" %user_keyword)

else:
    s_user=tuple(search_tweet["user_id"].tolist())
    params = {'l': s_user}

    cur.execute('SELECT * FROM users_data where user_id in %(l)s',params)
    search_user = pd.DataFrame(cur.fetchall())
    search_user.columns =  [i[0] for i in cur.description]

    search_full = pd.merge(search_tweet, search_user, how = 'left', on = ['user_id']).drop(['_id','id','user_id'], axis=1)
    search_full

In [ ]:
## search hashtag

user_keyword = input()
regx = re.compile(user_keyword, re.IGNORECASE)

hashtag_tweet = pd.DataFrame(
    db.tweet.find({"hashtags":{"$in": [regx]}}).sort([("retweet_count", -1),("reply_count", -1),("favorite_count", -1)])
)

if(len(hashtag_tweet)==0):
    print("Error: No result for %s" %user_keyword)

else:
    h_user=tuple(hashtag_tweet["user_id"].tolist())
    params = {'l': q_user}

    cur.execute('SELECT * FROM users_data where user_id in %(l)s',params)
    hash_user = pd.DataFrame(cur.fetchall())
    hash_user.columns =  [i[0] for i in cur.description]

    hash_search_table = pd.merge(hashtag_tweet, hash_user, how = 'left', on = ['user_id']).drop(['_id','id','user_id'], axis=1)
    hash_search_table

In [34]:
%%time
# using MongoDB
# most popular original tweet ranked by the retweet_count, reply_count, and favorite_count 
pop_tweet = pd.DataFrame(tweet.find({},{'user_id':1,'created_at':1,'text':1,'retweet_count':1,'favorite_count':1,'reply_count':1,'hashtags':1}).sort([("retweet_count", -1),("reply_count", -1),("favorite_count", -1)]))
pop_tweet

CPU times: user 178 ms, sys: 16 ms, total: 194 ms
Wall time: 429 ms


,_id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea779b8a8a689226877756a,1100261477989126145,"March 13, 2020, 12:43:40 AM",*corona virus enters my body*\n\nThe 4 Flintst...,237307,811062,1811,[]
1,5ea7792ba8a6892268776eb9,1112592502727548928,"March 18, 2020, 05:51:18 PM",When this Corona shit passes we have to promis...,181584,764405,1595,[]
...,...,...,...,...,...,...,...,...
19074,5ea77cc4a8a6892268779e78,3130596629,"April 25, 2020, 02:48:37 PM",We have 1/4 of all deaths from #Corona - thank...,0,0,0,"[Corona, ashamed]"
19075,5ea77cc4a8a6892268779e79,2279976427,"April 25, 2020, 02:48:38 PM",@VSReddy_MP I think wantedly you ate damaging ...,0,0,0,[]


In [35]:
# TOP 10 most popular hashtag ranked by aggregating the lists of hashtags (case insensitive)
from bson.son import SON
pipeline = [
     {"$unwind": "$hashtags"},
     {"$group": {"_id": {"$toLower": "$hashtags"}, "count": {"$sum": 1}}},
     {"$sort": SON([("count", -1), ("_id", -1)])}
] 

import pprint
pprint.pprint(list(db.tweet.aggregate(pipeline))[0:10])

[{'_id': 'corona', 'count': 1248},
 {'_id': 'covid19', 'count': 542},
 {'_id': 'coronavirus', 'count': 384},
 {'_id': 'covid_19', 'count': 292},
 {'_id': 'lockdown', 'count': 166},
 {'_id': 'covid', 'count': 129},
 {'_id': 'indiafightscorona', 'count': 119},
 {'_id': 'संतरामपालजी_का_सत्संग_सुनें', 'count': 114},
 {'_id': 'stayhome', 'count': 102},
 {'_id': 'covidー19', 'count': 72}]


Implement 2

In [36]:
# using SQL
# the most active verified user ranked by statuses_count
active_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY statuses_count DESC")
active_user.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,47596019,Liputan6.com,liputan6dotcom,True,3687560,693,1381273,Jakarta Indonesia,7062
1,16973333,The Independent,Independent,True,3202840,1146,976895,"London, England",3
2,378809160,𝔾𝕀𝔻𝕀𝕋ℝ𝔸𝔽𝔽𝕀ℂ,Gidi_Traffic,True,1676895,5826,922265,Everywhere,14478
3,2475273985,Paytm Care,Paytmcare,True,248675,15,861933,India,402
4,15448383,Inquirer,inquirerdotnet,True,3076056,1852,858348,Makati City,34260


In [38]:
# the most popular verified user ranked by followers_count
pop_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY followers_count DESC")
pop_user.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,18839785,Narendra Modi,narendramodi,True,55786179,2364,26559,India,0
1,807095,The New York Times,nytimes,True,46361159,904,393823,New York City,18483
2,145125358,Amitabh Bachchan,SrBachchan,True,41596464,1833,66428,"Mumbai, India",75
3,101311381,Shah Rukh Khan,iamsrk,True,40026738,77,58040,None,32
4,471741741,PMO India,PMOIndia,True,34461808,486,25996,India,0


In [39]:
# the most active location ranked by tweets count
pop_location = create_pandas_table("SELECT user_location, count(*) as amount FROM users_data WHERE user_location != 'None' GROUP BY user_location ORDER BY amount DESC")
pop_location.head(5)

,user_location,amount
0,India,363
1,"New Delhi, India",185
2,United States,125
3,"Mumbai, India",121
4,"Lagos, Nigeria",112






Join the tweet table and user table

In [40]:
%%time
# using Pandas
full_table = pd.merge(tweet_table, user_table, how = 'left', on = ['user_id'])
full_table

CPU times: user 44 ms, sys: 10.5 ms, total: 54.5 ms
Wall time: 66.8 ms


,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,5ea7785aa8a68922687753f6,1253949413191344128,207809313,"April 25, 2020, 07:30:12 AM",India’s war with Corona is ongoing.\n\nPlay yo...,398,2144,108,[FeedTheNeedy],BJP,BJP4India,True,13483759,3,200328,"6-A, Deen Dayal Upadhyay Marg,",0
1,5ea7785aa8a68922687753f7,1253992905703862272,858859031464751104,"April 25, 2020, 10:23:01 AM",VIDEO | 25.04.20\n\n&gt; Monsta X appears on Y...,59,144,1,"[몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]",Monsta X Europe,MonstaXEurope,False,27084,463,38058,let the haters bark,32020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19074,5ea77cc4a8a6892268779e78,1254059747738816512,3130596629,"April 25, 2020, 02:48:37 PM",We have 1/4 of all deaths from #Corona - thank...,0,0,0,"[Corona, ashamed]",Arkansas Boy,ArkansasWatch,False,276,867,4411,#Ark #LittleRock,1128
19075,5ea77cc4a8a6892268779e79,1254059751379361793,2279976427,"April 25, 2020, 02:48:38 PM",@VSReddy_MP I think wantedly you ate damaging ...,0,0,0,[],Venugopi,Venu7630,False,3,35,42,None,83


Implement 3

In [41]:
%%time

# using Pandas
# search the tweets that contain the keyword 'election'
election = full_table[full_table['text'].str.contains('election',case=False)].drop(['_id','id','user_id'], axis=1)

# rank the tweets that contain the keyword 'election' by favorite_count, retweet_count and reply_count
election.sort_values(by = ['favorite_count', 'retweet_count', 'reply_count'], ascending = (False, False, False))

CPU times: user 129 ms, sys: 6.03 ms, total: 135 ms
Wall time: 147 ms


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
4985,"April 06, 2020, 11:31:23 AM",The Democrats’ all-mail ballots proposal is a ...,5430,11868,1806,[],Ronna McDaniel,GOPChairwoman,True,602330,2594,14158,None,7107
4235,"April 25, 2020, 12:51:03 PM",Milwaukee’s health commissioner has now tied 4...,1046,1204,76,[],Ben Wikler,benwikler,True,132408,3960,31530,"Wisconsin, USA",58028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18955,"April 25, 2020, 02:47:46 PM",Every single person who was involved with this...,0,0,0,[],EmmaSofia,emmasofia3,False,162,536,16842,Gaia,4432
19071,"April 25, 2020, 02:48:36 PM",40 Coronavirus Cases In Milwaukee County Linke...,0,0,0,[],Jess Piper,EnglishTeach07,False,5026,5369,22781,Northwest MO,94498


In [42]:
%%time
## user interactive search for text (case insensitive)
search_keyword = input()
user_search = full_table[full_table['text'].str.contains(search_keyword,case=False)].drop(['_id','id','user_id'], axis=1)
user_search.sort_values(by = ['favorite_count', 'retweet_count', 'reply_count'], ascending = (False, False, False))

cancel
CPU times: user 116 ms, sys: 4.91 ms, total: 121 ms
Wall time: 9.94 s


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
524,"April 25, 2020, 03:31:42 AM",Corona? cancelled https://t.co/XHpTaCNC3t,347,4447,122,[],aksel,aksually,False,94104,605,5941,he/him,6403
6701,"April 24, 2020, 11:43:30 AM",Today my beautiful daughter turns 11 🥳🧁 \nI ha...,137,1468,677,[],𝐒𝐡𝐚𝐧𝐧𝐨𝐧 𝐢𝐧 𝐑𝐞𝐚𝐥 𝐋𝐢𝐟𝐞 ♡,Im_shaleelee,False,4735,1287,9729,None,16781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18399,"April 25, 2020, 02:43:01 PM",When you realize corona can't \ncancel your ex...,0,0,0,[],@meme_se_marunga,Memewala11,False,0,27,368,None,8
18979,"April 25, 2020, 02:47:54 PM","@DIPRRajasthan Sir ,Best regards, due to Coron...",0,0,0,[],Roop Chand Jain,RoopCha58911462,False,3,83,54,None,35


Implement 4

In [43]:
%%time
# using Pandas
# search for a hashtag 'lockdown' (case insensitive)
target = ['lockdown']
lockdown = full_table[pd.DataFrame(full_table.hashtags.tolist()).applymap(lambda s:s.lower() if type(s) == str else s).isin(target).any(1)].drop(['_id','id','user_id'], axis=1)

# rank the tweets that contain the hashtag 'lockdown' by followers_count
lockdown.sort_values(by = ['followers_count'], ascending = (False))

CPU times: user 257 ms, sys: 10.1 ms, total: 267 ms
Wall time: 274 ms


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
285,"April 21, 2020, 01:28:03 PM",My wholehearted thanks&amp; heartfelt felt ack...,2020,7103,41,"[Corona, Lockdown, JanaSeva]",Pawan Kalyan,PawanKalyan,True,3958111,161,1410,None,2
2210,"April 25, 2020, 06:17:11 AM",Scenarios at Koyambedu market !! Social distan...,16,80,10,"[lockdown, CoronaUpdates, corona]",IndiaGlitz - Tamil,igtamil,True,951444,666,68534,"Chennai, India",11776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18963,"April 25, 2020, 02:47:49 PM",Nice efforts guys..keep up the gud work✌\n#EIA...,0,0,0,"[EIAForGreenIndia, covid, lockdown, Corona, EI...",Aarti,Aarti03524188,False,0,12,24,None,17
14087,"April 25, 2020, 11:30:20 AM","How can Police officials can be so brutal, th...",1,0,0,"[DelhiPolice, delhi, PoliceBrutality, childabu...",Abbas Kazimi,AbbasKazimi7,False,0,46,23,"नई दिल्ली, भारत",80


In [44]:
%%time
## user interactive search for hashtag (case insensitive)

search_keyword = input()

target = [search_keyword]
lockdown = full_table[pd.DataFrame(full_table.hashtags.tolist()).applymap(lambda s:s.lower() if type(s) == str else s).isin(target).any(1)].drop(['_id','id','user_id'], axis=1)
lockdown.sort_values(by = ['followers_count'], ascending = (False))

remote
CPU times: user 251 ms, sys: 6.4 ms, total: 258 ms
Wall time: 10.2 s


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count


Implement 5

In [45]:
# Time
# We got the tweet data and corresponding user data in one time slot, but we save the data based on the original tweet
# try to search the earliest created original tweet that is still getting retweeted in the time slot.

In [46]:
%%time
full_table['created_at'] = pd.to_datetime(full_table['created_at'])
full_table.sort_values(by = ['created_at'], ascending = (True))

CPU times: user 2.81 s, sys: 9.23 ms, total: 2.82 s
Wall time: 2.86 s


,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
14241,5ea77b71a8a6892268778b97,341578219176214528,1242575784,2013-06-03 15:32:48,Corona virus....its coming,73776,131225,9968,[],Marco,Marco_Acortes,False,5652,106,3839,None,4842
6794,5ea77927a8a6892268776e80,872849401928179713,189011442,2017-06-08 16:14:48,"But to return to a moment of grace.\n\n""Take m...",16,39,0,[],coldplums,coldplums,False,4341,1435,24661,on the road,41167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19073,5ea77cc4a8a6892268779e77,1254059746652389378,734263483173212160,2020-04-25 14:48:37,i will be makin tacos and gettin drunk on cinc...,0,0,0,[],dirty d 😽💙,deannarae21,False,684,482,43705,baerobbie 💙,75847
19075,5ea77cc4a8a6892268779e79,1254059751379361793,2279976427,2020-04-25 14:48:38,@VSReddy_MP I think wantedly you ate damaging ...,0,0,0,[],Venugopi,Venu7630,False,3,35,42,None,83
